# Cuaderno para consumir de Presto

En este cuaderno se muestra un ejemplo para consumir de l servicio de presto una vez  que los datos esten cargados a MinIO y las tbalas definidas en Hive-Metastore.


## Librerias

In [ ]:
import prestodb
import pandas as pd

## Definición de la coexión

In [ ]:
conn=prestodb.dbapi.connect(
    host='presto',
    port=8080,
    user='jupyter',
    catalog='hive',
    schema='default',
)

## Ejemplo 

Podemos definir las consultas como sql y obtener las tablas.
En este ejemplo se seleccionan todos los datos de una tabla

In [ ]:
cur = conn.cursor()
cur.execute('SELECT * FROM conjunto_de_datos_erogaciones_enigh_2018_ns')
rows = cur.fetchall()

# El nombre de las columnas no se regresa al hacer la consulta por lo que se hace una consulta especial para estos 
cur = conn.cursor()
cur.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'conjunto_de_datos_erogaciones_enigh_2018_ns'")
columns = cur.fetchall()

pd.DataFrame(rows,columns=[x[0] for x in columns])

Podemos tambien obterner todos los nombres de las tablas definidas en el schema

In [ ]:
cur.execute("SHOW TABLES")
tables = cur.fetchall()
tables

En este ejemplo hacemos la consulta de la poblacion total para 2018. La cual coincide con la tabla oficial publicada en 

https://www.inegi.org.mx/contenidos/programas/enigh/nc/2018/tabulados/enigh2018_ns_basicos_tabulados.xlsx

In [ ]:
cur.execute(""" 
SELECT SUM(INTEGRANTES*FACTOR) AS POBLACION
FROM conjunto_de_datos_vivienda_enigh_2018_ns AS VIV
INNER JOIN (
    SELECT FOLIOVIV, COUNT(*) AS INTEGRANTES
    FROM conjunto_de_datos_poblacion_enigh_2018_ns 
    GROUP BY FOLIOVIV
) POB 
ON VIV.FOLIOVIV = POB.FOLIOVIV
""")
RES = cur.fetchall()

"{:,}".format(int(RES[0][0]))

LA información anterior puede ser desglosada por grupos de edad 

In [ ]:
cur.execute(""" 
SELECT G_EDAD, SUM(INTEGRANTES*FACTOR) AS POBLACION
FROM conjunto_de_datos_vivienda_enigh_2018_ns AS VIV
INNER JOIN (
    SELECT FOLIOVIV, G_EDAD, COUNT(*) AS INTEGRANTES
    FROM (SELECT FOLIOVIV, CASE 
            WHEN EDAD < 5 THEN '00-04 AÑOS'
            WHEN EDAD BETWEEN 5 AND 11 THEN '05-11 AÑOS'
            WHEN EDAD BETWEEN 12 AND 19 THEN '12-19 AÑOS'
            WHEN EDAD BETWEEN 20 AND 29 THEN '20-29 AÑOS'
            WHEN EDAD BETWEEN 30 AND 39 THEN '30-39 AÑOS'
            WHEN EDAD BETWEEN 40 AND 49 THEN '40-49 AÑOS'
            WHEN EDAD BETWEEN 50 AND 59 THEN '50-59 AÑOS'
            WHEN EDAD > 59  THEN '60 Y MÁS AÑOS'
            END AS G_EDAD
        FROM conjunto_de_datos_poblacion_enigh_2018_ns 
    )
    GROUP BY FOLIOVIV, G_EDAD
) POB 
ON VIV.FOLIOVIV = POB.FOLIOVIV
GROUP BY G_EDAD
""")
RES2 = cur.fetchall()

In [ ]:
datos = pd.DataFrame(RES2, columns = ["G_EDAD","POBLACION"])
datos["formato"] = datos["POBLACION"].astype(int).apply(lambda x: "{:,}".format(x))
datos.sort_values("G_EDAD")